In [ ]:
#Extract KI67 signal from segmented tiled images

In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import os
import fnmatch
import skimage
import cv2

In [11]:
redPath = '/Volumes/Aortas/Aorta/preprocessedImages/KI67/maxIP/P7/aorta4'
nucleiPath = '/Volumes/Aortas/Aorta/segmentations/KI67/tiled/nuclei/P7/aorta4'

In [12]:
foldersRed = [x[0] for x in os.walk(redPath)]
foldersRed = sorted(foldersRed)

foldersNuclei = [x[0] for x in os.walk(nucleiPath)]
foldersNuclei = sorted(foldersNuclei)

In [13]:
df = pd.DataFrame()
for folder in foldersNuclei:
    df = pd.DataFrame()
    filesNuclei = [f for f in listdir(folder) if isfile(join(folder, f)) if "_Alx568_seg.npy" in f if "._" not in f]
    aorta = folder.split("/")[-1]
    age = folder.split("/")[-2]
    original = folder.split("/")[-3]

    for fileNuclei in filesNuclei:
        nuclei = np.load(join(folder,fileNuclei), allow_pickle = True).item()

        fileRed = fileNuclei.split("_Alx568_seg.npy")[0] + "_Alx647_1.tif"
        #folderRed = join(redPath, original, age, aorta)
        folderRed = join(redPath)
        redImage = skimage.io.imread(join(folderRed, fileRed))
        redImageNorm = cv2.normalize(redImage, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

        nucleiMasks = nuclei['masks']
        nucleiMaskRedInt = np.where(nucleiMasks > 0, redImageNorm, 0 )
        medianBackground = np.median(redImage[np.where(nucleiMasks == 0)])
        medianBackgroundNorm = np.median(redImageNorm[np.where(nucleiMasks == 0)])
        meanBackground = np.mean(redImage[np.where(nucleiMasks == 0)])
        meanBackgroundNorm = np.mean(redImageNorm[np.where(nucleiMasks == 0)])

        for i in np.unique(nucleiMasks):
            if i != 0:
                median = np.median(redImage[np.where(nucleiMasks == i)])
                medianNorm = np.median(redImageNorm[np.where(nucleiMasks == i)])
                mean = np.mean(redImage[np.where(nucleiMasks == i)])
                meanNorm = np.mean(redImageNorm[np.where(nucleiMasks == i)])

                dict = {'original': original, 'age': age, 'aorta': aorta, 'median': median, 'mean': mean, 'medianNorm': medianNorm, 'meanNorm': meanNorm, 'medianBackground': medianBackground, 'meanBackground': meanBackground, 'medianBackgroundNorm': medianBackgroundNorm, 'meanBackgroundNorm': meanBackgroundNorm,  'file': fileNuclei.split("c3_seg.npy")[0]}
                dfData = pd.DataFrame(dict, index= [0])
                df = pd.concat((df, dfData), axis=0, ignore_index=True)

    filename = "coordinatesKI67Intensity_" + original + "_" + age + "_" + aorta + "_Tiled.csv"
    df.to_csv(join("/Volumes/Aortas/Aorta/results/KI67/coordinatesKI67",filename), index = False)

OSError: Failed to interpret file '/Volumes/Aortas/Aorta/segmentations/KI67/tiled/nuclei/P7/aorta4/._220901_Ki67_P7 aorta 4-MaxIP_#4_Alx568_seg.npy' as a pickle